In [9]:
from sklearn import metrics
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
import data_preparation
import pandas as pd
import numpy as np

### Path to test dataset (csv format)

In [10]:
csv_path = 'weather.csv'

In [11]:
df = data_preparation.prepare_data(csv_path, "test_dataset.csv")
valid_ts, y_valid, n_input, n_features  = data_preparation.preprocess(data=df, train_model=False)

In [22]:
regression_columns = [
 'M_RAIN_PERCENTAGE_5',
 'M_RAIN_PERCENTAGE_10',
'M_RAIN_PERCENTAGE_15',
 'M_RAIN_PERCENTAGE_30',
 'M_RAIN_PERCENTAGE_45',
 'M_RAIN_PERCENTAGE_60',
]
classification_columns = [
'M_WEATHER_FORECAST_SAMPLES_M_WEATHER_5',
'M_WEATHER_FORECAST_SAMPLES_M_WEATHER_10',
'M_WEATHER_FORECAST_SAMPLES_M_WEATHER_15',
'M_WEATHER_FORECAST_SAMPLES_M_WEATHER_30',
'M_WEATHER_FORECAST_SAMPLES_M_WEATHER_45',
'M_WEATHER_FORECAST_SAMPLES_M_WEATHER_60'
]
y_columns = [
    'M_RAIN_PERCENTAGE_5',
    'M_RAIN_PERCENTAGE_10',
    'M_RAIN_PERCENTAGE_15',
    'M_RAIN_PERCENTAGE_30',
    'M_RAIN_PERCENTAGE_45',
    'M_RAIN_PERCENTAGE_60',
    'M_WEATHER_FORECAST_SAMPLES_M_WEATHER_5',
    'M_WEATHER_FORECAST_SAMPLES_M_WEATHER_10',
    'M_WEATHER_FORECAST_SAMPLES_M_WEATHER_15',
    'M_WEATHER_FORECAST_SAMPLES_M_WEATHER_30',
    'M_WEATHER_FORECAST_SAMPLES_M_WEATHER_45',
    'M_WEATHER_FORECAST_SAMPLES_M_WEATHER_60',
    ]


In [17]:
model = tf.keras.models.load_model('model.h5')

In [19]:
y_pred = model.predict(valid_ts)

In [23]:
def regression_metrics(y_pred, ground_truth):
    predictions = pd.DataFrame(y_pred, columns = y_columns)
    pred_df = predictions[regression_columns]
    ground_truth = ground_truth[regression_columns]
    mae = metrics.mean_absolute_error(ground_truth.to_numpy()[n_input:], pred_df.to_numpy())
    mse = metrics.mean_squared_error(ground_truth.to_numpy()[n_input:], pred_df.to_numpy())
    rmse = np.sqrt(mse)
    return mae, mse, rmse

def accuracy(y_pred, ground_truth):
    predictions = pd.DataFrame(y_pred,columns=y_columns)
    pred_df = predictions[classification_columns].astype(int)
    equality = tf.math.equal(pred_df.to_numpy(), ground_truth[classification_columns].iloc[:len(ground_truth) - n_input].to_numpy().astype(int))
    return tf.math.reduce_mean(tf.cast(equality, tf.float32)).numpy()

In [24]:
_accuracy = accuracy(y_pred, y_valid)
mae,mse,rmse = regression_metrics(y_pred, y_valid)

In [28]:
print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'Accuracy: {_accuracy}')

MAE: 1.435493593933331
MSE: 14.204162071424335
RMSE: 3.768840945360302
Accuracy: 0.8246849775314331


In [25]:
def get_model_output(y_pred):
    preds_dicts = []
    predictions = pd.DataFrame(y_pred,columns=y_columns)
    predictions[classification_columns] = predictions[classification_columns].astype(int)
    for _, row in predictions.iterrows():
        preds_dicts.append({
            '5': {
                'type': int(row['M_WEATHER_FORECAST_SAMPLES_M_WEATHER_5']),
                'rain_percentage': row['M_RAIN_PERCENTAGE_5']
            },
            '10': {
                'type': int(row['M_WEATHER_FORECAST_SAMPLES_M_WEATHER_10']),
                'rain_percentage': row['M_RAIN_PERCENTAGE_10']
            },
            '15': {
                'type': int(row['M_WEATHER_FORECAST_SAMPLES_M_WEATHER_15']),
                'rain_percentage': row['M_RAIN_PERCENTAGE_15']
            },
            '30': {
                'type': int(row['M_WEATHER_FORECAST_SAMPLES_M_WEATHER_30']),
                'rain_percentage': row['M_RAIN_PERCENTAGE_30']
            },
            '45': {
                'type': int(row['M_WEATHER_FORECAST_SAMPLES_M_WEATHER_45']),
                'rain_percentage': row['M_RAIN_PERCENTAGE_45']
            },
            '60': {
                'type': int(row['M_WEATHER_FORECAST_SAMPLES_M_WEATHER_60']),
                'rain_percentage': row['M_RAIN_PERCENTAGE_60']
            },
        })
    return preds_dicts

In [26]:
results = get_model_output(y_pred)

In [27]:
results

[{'5': {'type': 0, 'rain_percentage': 2.8392703533172607},
  '10': {'type': 0, 'rain_percentage': 2.819629430770874},
  '15': {'type': 0, 'rain_percentage': 2.8020269870758057},
  '30': {'type': 0, 'rain_percentage': 4.076732635498047},
  '45': {'type': 0, 'rain_percentage': 4.565898895263672},
  '60': {'type': 0, 'rain_percentage': 3.7921395301818848}},
 {'5': {'type': 0, 'rain_percentage': 2.839083433151245},
  '10': {'type': 0, 'rain_percentage': 2.8194632530212402},
  '15': {'type': 0, 'rain_percentage': 2.8018617630004883},
  '30': {'type': 0, 'rain_percentage': 4.076619625091553},
  '45': {'type': 0, 'rain_percentage': 4.565619468688965},
  '60': {'type': 0, 'rain_percentage': 3.791598081588745}},
 {'5': {'type': 0, 'rain_percentage': 2.838895320892334},
  '10': {'type': 0, 'rain_percentage': 2.81929874420166},
  '15': {'type': 0, 'rain_percentage': 2.801696538925171},
  '30': {'type': 0, 'rain_percentage': 4.076508522033691},
  '45': {'type': 0, 'rain_percentage': 4.565343379974